# 🍎 파이썬 머신러닝 완벽 가이드 혼공

### 2019.04.01 ~ 2019.04.11 교재 03-05장

03. 평가 
04. 분류 
05. 회귀

### 분류 실습 - 캐글 산탄데르 고객 만족 예측 

고객만족 여부를 XGBoost와 LightGBM을 활용해 예측할 것임.

#### XGBoost

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb

In [2]:
train = pd.read_csv('data/04. Santander_train.csv')
train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [3]:
test = pd.read_csv('data/04. Santander_test.csv')
test.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.10
1,5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.72
2,6,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.95
3,7,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.61
4,9,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.73


In [4]:
train.shape

(76020, 371)

아래 info()를 보면, null 값이 존재하지 않다는 것을 알 수 있음.

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


그럼 타겟값이 뭔지를 알아야지. 타겟값을 예측해야 하는 것이니까. 
그리고 타겟값이 어떤 비율로 되어 있는지도 함께 보는게 좋을 듯

In [6]:
train['TARGET'].value_counts()

0    73012
1     3008
Name: TARGET, dtype: int64

In [7]:
unsatisfied_cnt = train[train['TARGET'] == 1]['TARGET'].count()
unsatisfied_cnt

3008

In [8]:
total_cnt = train[train['TARGET'] == 0]['TARGET'].count()
total_cnt

73012

In [9]:
unsatisfied_cnt / total_cnt

0.04119870706185284

대략 0.4%의 사람들이 불만족한다고 보면 됨. 그러면 예측한 값도 불만족한 것들이 나와야 겠지!

In [10]:
train.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


보면 var3 칼럼의 경우 최소 값이 -999999니까 이렇게 되면 당연히 성능이 좋아지지 않겠지. 116개나 있으니까, 편차가 심해져서 가장 값이 많은 2로 변환을 해줄 것! 

In [11]:
train[train['var3'] == -999999.000000]['var3'].count()

116

In [12]:
train['var3'].replace(-999999.000000, 2, inplace=True)


In [13]:
train.head() # 잘 바뀌었고! 

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [14]:
# train.drop('ID',1, inplace=True)  # ID는 중요한 컬럼이 아니라 삭제! 
train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [15]:
X_features = train.iloc[:,:-1] # 마지막 target을 없앰 
X_features

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76015,151829,2,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60926.490000
76016,151830,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118634.520000
76017,151835,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74028.150000
76018,151836,2,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84278.160000


In [16]:
y_labels = train.iloc[:,-1]  # target 값만 따로 두는 것! 
y_labels

0        0
1        0
2        0
3        0
4        0
        ..
76015    0
76016    0
76017    0
76018    0
76019    0
Name: TARGET, Length: 76020, dtype: int64

이 데이터는 불만족률이 0.4이므로 비대칭적 데이터임. 그러므로 학습 데이터와 테스트 데이터 세트 모두 target의 분포가 0.4와 비슷해야지 올바른 예측이 가능해지는 것!

In [17]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=0)

In [18]:
train_cnt = y_train.count()
train_cnt

60816

In [19]:
test_cnt = y_test.count()
test_cnt

15204

In [20]:
y_train.value_counts() / train_cnt

0    0.960964
1    0.039036
Name: TARGET, dtype: float64

In [21]:
y_test.value_counts() / test_cnt

0    0.9583
1    0.0417
Name: TARGET, dtype: float64

In [23]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# n_estimators는 500으로, random state는 예제 수행 시마다 동일 예측 결과를 위해 설정. 
xgb_clf = XGBClassifier(n_estimators=500, random_state=156)

# 성능 평가 지표를 auc로, 조기 중단 파라미터는 100으로 설정하고 학습 수행. 
xgb_clf.fit(X_train, y_train, early_stopping_rounds=100,
            eval_metric="auc", eval_set=[(X_train, y_train), (X_test, y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.799928	validation_1-auc:0.803548
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.802222	validation_1-auc:0.805222
[2]	validation_0-auc:0.80819	validation_1-auc:0.813162
[3]	validation_0-auc:0.8127	validation_1-auc:0.813243
[4]	validation_0-auc:0.81648	validation_1-auc:0.816979
[5]	validation_0-auc:0.816018	validation_1-auc:0.816629
[6]	validation_0-auc:0.816474	validation_1-auc:0.817776
[7]	validation_0-auc:0.818148	validation_1-auc:0.818464
[8]	validation_0-auc:0.81806	validation_1-auc:0.818295
[9]	validation_0-auc:0.817039	validation_1-auc:0.818087
[10]	validation_0-auc:0.818318	validation_1-auc:0.818749
[11]	validation_0-auc:0.818711	validation_1-auc:0.818521
[12]	validation_0-auc:0.818673	validation_1-auc:0.818516
[13]	validation_0-auc:0.819156	validation_1-auc:0.818998
[14]	validation_0-auc:0.819847	validation_1-auc:0.81999
[15

[141]	validation_0-auc:0.863144	validation_1-auc:0.841617
[142]	validation_0-auc:0.863281	validation_1-auc:0.841652
[143]	validation_0-auc:0.863379	validation_1-auc:0.841669
[144]	validation_0-auc:0.863425	validation_1-auc:0.841631
[145]	validation_0-auc:0.863468	validation_1-auc:0.841577
[146]	validation_0-auc:0.863516	validation_1-auc:0.841604
[147]	validation_0-auc:0.863612	validation_1-auc:0.84166
[148]	validation_0-auc:0.863779	validation_1-auc:0.841698
[149]	validation_0-auc:0.864096	validation_1-auc:0.841887
[150]	validation_0-auc:0.864215	validation_1-auc:0.841922
[151]	validation_0-auc:0.864265	validation_1-auc:0.841935
[152]	validation_0-auc:0.864337	validation_1-auc:0.842071
[153]	validation_0-auc:0.864482	validation_1-auc:0.842172
[154]	validation_0-auc:0.864563	validation_1-auc:0.842284
[155]	validation_0-auc:0.864651	validation_1-auc:0.842306
[156]	validation_0-auc:0.864725	validation_1-auc:0.842335
[157]	validation_0-auc:0.864743	validation_1-auc:0.842308
[158]	validatio

헉, 이거 완전 느림.... 이게 그래도 조기 중단으로 시간을 줄인건데도 시간이 많이 걸리네. 이래서 랜포가 자주 이용되구나~

In [24]:
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators를 100으로 감소
xgb_clf = XGBClassifier(n_estimators=100)

params = {'max_depth':[5, 7] , 'min_child_weight':[1,3] ,'colsample_bytree':[0.5, 0.75] }

# 하이퍼 파라미터 테스트의 수행속도를 향상 시키기 위해 cv 를 지정하지 않음. 
gridcv = GridSearchCV(xgb_clf, param_grid=params)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc",
           eval_set=[(X_train, y_train), (X_test, y_test)])

print('GridSearchCV 최적 파라미터:',gridcv.best_params_) 

xgb_roc_score = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

/Users/harampark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[0]	validation_0-auc:0.794403	validation_1-auc:0.795121
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.807257	validation_1-auc:0.80411
[2]	validation_0-auc:0.81043	validation_1-auc:0.806105
[3]	validation_0-auc:0.821538	validation_1-auc:0.816025
[4]	validation_0-auc:0.826857	validation_1-auc:0.824696
[5]	validation_0-auc:0.829313	validation_1-auc:0.826868
[6]	validation_0-auc:0.830171	validation_1-auc:0.8254
[7]	validation_0-auc:0.831324	validation_1-auc:0.826348
[8]	validation_0-auc:0.833191	validation_1-auc:0.829628
[9]	validation_0-auc:0.83424	validation_1-auc:0.830034
[10]	validation_0-auc:0.836539	validation_1-auc:0.829993
[11]	validation_0-auc:0.835097	validation_1-auc:0.831106
[12]	validation_0-auc:0.832193	validation_1-auc:0.828385
[13]	validation_0-auc:0.836263	validation_1-auc:0.830918
[14]	validation_0-auc:0.83496	validation_1-auc:0.829852
[15]

[66]	validation_0-auc:0.870802	validation_1-auc:0.844856
[67]	validation_0-auc:0.871206	validation_1-auc:0.844562
[68]	validation_0-auc:0.871583	validation_1-auc:0.844526
[69]	validation_0-auc:0.87194	validation_1-auc:0.844413
[70]	validation_0-auc:0.8723	validation_1-auc:0.844269
[71]	validation_0-auc:0.872782	validation_1-auc:0.84385
[72]	validation_0-auc:0.873082	validation_1-auc:0.844116
[73]	validation_0-auc:0.873354	validation_1-auc:0.844138
[74]	validation_0-auc:0.873613	validation_1-auc:0.844083
[75]	validation_0-auc:0.873759	validation_1-auc:0.844172
[76]	validation_0-auc:0.873939	validation_1-auc:0.844256
[77]	validation_0-auc:0.874137	validation_1-auc:0.844127
[78]	validation_0-auc:0.874318	validation_1-auc:0.844047
[79]	validation_0-auc:0.874543	validation_1-auc:0.844264
[80]	validation_0-auc:0.874706	validation_1-auc:0.844016
[81]	validation_0-auc:0.875014	validation_1-auc:0.844037
[82]	validation_0-auc:0.875394	validation_1-auc:0.844052
[83]	validation_0-auc:0.875528	vali

[18]	validation_0-auc:0.838705	validation_1-auc:0.833702
[19]	validation_0-auc:0.840418	validation_1-auc:0.8343
[20]	validation_0-auc:0.839365	validation_1-auc:0.834171
[21]	validation_0-auc:0.841713	validation_1-auc:0.836117
[22]	validation_0-auc:0.84148	validation_1-auc:0.835511
[23]	validation_0-auc:0.84388	validation_1-auc:0.837513
[24]	validation_0-auc:0.845339	validation_1-auc:0.837993
[25]	validation_0-auc:0.846105	validation_1-auc:0.838786
[26]	validation_0-auc:0.846682	validation_1-auc:0.838678
[27]	validation_0-auc:0.848078	validation_1-auc:0.838604
[28]	validation_0-auc:0.847858	validation_1-auc:0.838976
[29]	validation_0-auc:0.849101	validation_1-auc:0.839864
[30]	validation_0-auc:0.850738	validation_1-auc:0.841243
[31]	validation_0-auc:0.851649	validation_1-auc:0.841185
[32]	validation_0-auc:0.852759	validation_1-auc:0.842192
[33]	validation_0-auc:0.853854	validation_1-auc:0.842161
[34]	validation_0-auc:0.854579	validation_1-auc:0.841541
[35]	validation_0-auc:0.854959	vali

Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.821408	validation_1-auc:0.820387
[2]	validation_0-auc:0.825965	validation_1-auc:0.820638
[3]	validation_0-auc:0.829803	validation_1-auc:0.823783
[4]	validation_0-auc:0.831099	validation_1-auc:0.828165
[5]	validation_0-auc:0.832542	validation_1-auc:0.830075
[6]	validation_0-auc:0.833575	validation_1-auc:0.830488
[7]	validation_0-auc:0.834311	validation_1-auc:0.83084
[8]	validation_0-auc:0.837343	validation_1-auc:0.830965
[9]	validation_0-auc:0.838531	validation_1-auc:0.830997
[10]	validation_0-auc:0.838972	validation_1-auc:0.831321
[11]	validation_0-auc:0.839419	validation_1-auc:0.832468
[12]	validation_0-auc:0.840527	validation_1-auc:0.832764
[13]	validation_0-auc:0.841071	validation_1-auc:0.833057
[14]	validation_0-auc:0.840627	validation_1-auc:0.83299
[15]	validation_0-auc:0.839588	validation_1-auc:0.831533

[48]	validation_0-auc:0.885056	validation_1-auc:0.835361
[49]	validation_0-auc:0.885423	validation_1-auc:0.834958
[50]	validation_0-auc:0.886467	validation_1-auc:0.83518
[51]	validation_0-auc:0.886695	validation_1-auc:0.834737
[52]	validation_0-auc:0.887081	validation_1-auc:0.834871
[53]	validation_0-auc:0.887455	validation_1-auc:0.835115
[54]	validation_0-auc:0.887356	validation_1-auc:0.835132
[55]	validation_0-auc:0.887571	validation_1-auc:0.835145
[56]	validation_0-auc:0.888165	validation_1-auc:0.834806
[57]	validation_0-auc:0.888224	validation_1-auc:0.834483
[58]	validation_0-auc:0.888245	validation_1-auc:0.834344
[59]	validation_0-auc:0.888307	validation_1-auc:0.834241
[60]	validation_0-auc:0.888418	validation_1-auc:0.83439
[61]	validation_0-auc:0.88877	validation_1-auc:0.834087
[62]	validation_0-auc:0.888935	validation_1-auc:0.833934
[63]	validation_0-auc:0.889433	validation_1-auc:0.833982
[64]	validation_0-auc:0.890142	validation_1-auc:0.833956
Stopping. Best iteration:
[34]	val

[50]	validation_0-auc:0.886727	validation_1-auc:0.841442
[51]	validation_0-auc:0.886999	validation_1-auc:0.841215
[52]	validation_0-auc:0.887235	validation_1-auc:0.841138
[53]	validation_0-auc:0.887527	validation_1-auc:0.841529
[54]	validation_0-auc:0.887939	validation_1-auc:0.841707
[55]	validation_0-auc:0.888188	validation_1-auc:0.84211
[56]	validation_0-auc:0.888523	validation_1-auc:0.84199
[57]	validation_0-auc:0.888654	validation_1-auc:0.841944
[58]	validation_0-auc:0.888814	validation_1-auc:0.841815
[59]	validation_0-auc:0.888954	validation_1-auc:0.841745
[60]	validation_0-auc:0.889427	validation_1-auc:0.841826
[61]	validation_0-auc:0.889988	validation_1-auc:0.841887
[62]	validation_0-auc:0.890105	validation_1-auc:0.841674
[63]	validation_0-auc:0.890662	validation_1-auc:0.841734
[64]	validation_0-auc:0.89142	validation_1-auc:0.841494
[65]	validation_0-auc:0.891925	validation_1-auc:0.841368
[66]	validation_0-auc:0.892108	validation_1-auc:0.841256
[67]	validation_0-auc:0.892727	val

[40]	validation_0-auc:0.876255	validation_1-auc:0.843665
[41]	validation_0-auc:0.876671	validation_1-auc:0.84382
[42]	validation_0-auc:0.876864	validation_1-auc:0.8437
[43]	validation_0-auc:0.87741	validation_1-auc:0.843491
[44]	validation_0-auc:0.878334	validation_1-auc:0.843603
[45]	validation_0-auc:0.879145	validation_1-auc:0.843872
[46]	validation_0-auc:0.879441	validation_1-auc:0.843206
[47]	validation_0-auc:0.880253	validation_1-auc:0.843476
[48]	validation_0-auc:0.880603	validation_1-auc:0.843201
[49]	validation_0-auc:0.881143	validation_1-auc:0.842618
[50]	validation_0-auc:0.881782	validation_1-auc:0.842672
[51]	validation_0-auc:0.882094	validation_1-auc:0.842864
[52]	validation_0-auc:0.882537	validation_1-auc:0.842364
[53]	validation_0-auc:0.883137	validation_1-auc:0.84252
[54]	validation_0-auc:0.88329	validation_1-auc:0.842857
[55]	validation_0-auc:0.883918	validation_1-auc:0.842915
[56]	validation_0-auc:0.884552	validation_1-auc:0.842658
[57]	validation_0-auc:0.884666	valida

[8]	validation_0-auc:0.831075	validation_1-auc:0.82492
[9]	validation_0-auc:0.833736	validation_1-auc:0.826468
[10]	validation_0-auc:0.833919	validation_1-auc:0.82652
[11]	validation_0-auc:0.834924	validation_1-auc:0.82715
[12]	validation_0-auc:0.837146	validation_1-auc:0.82799
[13]	validation_0-auc:0.837153	validation_1-auc:0.82798
[14]	validation_0-auc:0.837844	validation_1-auc:0.829876
[15]	validation_0-auc:0.837892	validation_1-auc:0.829625
[16]	validation_0-auc:0.839263	validation_1-auc:0.830526
[17]	validation_0-auc:0.840856	validation_1-auc:0.832299
[18]	validation_0-auc:0.843002	validation_1-auc:0.834838
[19]	validation_0-auc:0.844208	validation_1-auc:0.835412
[20]	validation_0-auc:0.84495	validation_1-auc:0.836659
[21]	validation_0-auc:0.84609	validation_1-auc:0.83703
[22]	validation_0-auc:0.847128	validation_1-auc:0.837593
[23]	validation_0-auc:0.847677	validation_1-auc:0.8373
[24]	validation_0-auc:0.848415	validation_1-auc:0.837701
[25]	validation_0-auc:0.848778	validation_1

KeyboardInterrupt: 

와, 시간이 너무 오래걸려서 중단하고 다른 하이퍼 파라미터를 집어넣는 것은 생략

#### LightGBM 모델 학습과 하이퍼 파라미터 튜닝

In [29]:
from lightgbm import LGBMClassifier 

In [30]:
lgbm_clf = LGBMClassifier(n_estimators=500)

In [31]:
evals = [(X_test, y_test)]
lgbm_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="auc", eval_set=evals,
                verbose=True)

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

[1]	valid_0's auc: 0.817877	valid_0's binary_logloss: 0.165052
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.817734	valid_0's binary_logloss: 0.160082
[3]	valid_0's auc: 0.826796	valid_0's binary_logloss: 0.156395
[4]	valid_0's auc: 0.831011	valid_0's binary_logloss: 0.15354
[5]	valid_0's auc: 0.833917	valid_0's binary_logloss: 0.151312
[6]	valid_0's auc: 0.833856	valid_0's binary_logloss: 0.149504
[7]	valid_0's auc: 0.83643	valid_0's binary_logloss: 0.148079
[8]	valid_0's auc: 0.837348	valid_0's binary_logloss: 0.146815
[9]	valid_0's auc: 0.839167	valid_0's binary_logloss: 0.145641
[10]	valid_0's auc: 0.838244	valid_0's binary_logloss: 0.144916
[11]	valid_0's auc: 0.838277	valid_0's binary_logloss: 0.144231
[12]	valid_0's auc: 0.838228	valid_0's binary_logloss: 0.143605
[13]	valid_0's auc: 0.838763	valid_0's binary_logloss: 0.142999
[14]	valid_0's auc: 0.838946	valid_0's binary_logloss: 0.142441
[15]	valid_0's auc: 0.838022	valid_0's binary_logloss

[133]	valid_0's auc: 0.835725	valid_0's binary_logloss: 0.140925
[134]	valid_0's auc: 0.835667	valid_0's binary_logloss: 0.14094
[135]	valid_0's auc: 0.835552	valid_0's binary_logloss: 0.140988
[136]	valid_0's auc: 0.83548	valid_0's binary_logloss: 0.141023
[137]	valid_0's auc: 0.835283	valid_0's binary_logloss: 0.14108
[138]	valid_0's auc: 0.835301	valid_0's binary_logloss: 0.14108
[139]	valid_0's auc: 0.83509	valid_0's binary_logloss: 0.14114
[140]	valid_0's auc: 0.834976	valid_0's binary_logloss: 0.141178
[141]	valid_0's auc: 0.834828	valid_0's binary_logloss: 0.141238
Early stopping, best iteration is:
[41]	valid_0's auc: 0.839193	valid_0's binary_logloss: 0.139697
ROC AUC: 0.8392


시간이 확실히 덜 걸림! 

In [32]:
lgbm_clf = LGBMClassifier(n_estimators=200)
params = {'num_leaves': [32,64],
         'max_depth':[128,160],
         'min_child_samples':[60,100],
         'subsample':[0.8,1]}

gridcv = GridSearchCV(lgbm_clf, param_grid=params, cv=3)
gridcv.fit(X_train, y_train, early_stopping_rounds=30,
          eval_metric="auc", 
          eval_set=[(X_train, y_train), (X_test, y_test)])

gridcf.best_params_

[1]	valid_0's auc: 0.81963	valid_0's binary_logloss: 0.156025	valid_1's auc: 0.81308	valid_1's binary_logloss: 0.164955
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.8261	valid_0's binary_logloss: 0.150835	valid_1's auc: 0.821606	valid_1's binary_logloss: 0.159814
[3]	valid_0's auc: 0.83273	valid_0's binary_logloss: 0.147047	valid_1's auc: 0.827513	valid_1's binary_logloss: 0.156348
[4]	valid_0's auc: 0.838121	valid_0's binary_logloss: 0.144087	valid_1's auc: 0.832759	valid_1's binary_logloss: 0.153386
[5]	valid_0's auc: 0.841246	valid_0's binary_logloss: 0.141613	valid_1's auc: 0.835436	valid_1's binary_logloss: 0.151087
[6]	valid_0's auc: 0.844251	valid_0's binary_logloss: 0.139588	valid_1's auc: 0.836285	valid_1's binary_logloss: 0.149129
[7]	valid_0's auc: 0.84542	valid_0's binary_logloss: 0.137959	valid_1's auc: 0.837304	valid_1's binary_logloss: 0.147647
[8]	valid_0's auc: 0.847757	valid_0's binary_logloss: 0.136565	valid_1's auc: 0.838784	vali

[28]	valid_0's auc: 0.873841	valid_0's binary_logloss: 0.124233	valid_1's auc: 0.838885	valid_1's binary_logloss: 0.139549
[29]	valid_0's auc: 0.875275	valid_0's binary_logloss: 0.123891	valid_1's auc: 0.839588	valid_1's binary_logloss: 0.139395
[30]	valid_0's auc: 0.876347	valid_0's binary_logloss: 0.123559	valid_1's auc: 0.839956	valid_1's binary_logloss: 0.139294
[31]	valid_0's auc: 0.877439	valid_0's binary_logloss: 0.123251	valid_1's auc: 0.839595	valid_1's binary_logloss: 0.139275
[32]	valid_0's auc: 0.878516	valid_0's binary_logloss: 0.122929	valid_1's auc: 0.839623	valid_1's binary_logloss: 0.139268
[33]	valid_0's auc: 0.879362	valid_0's binary_logloss: 0.12267	valid_1's auc: 0.838978	valid_1's binary_logloss: 0.139321
[34]	valid_0's auc: 0.880143	valid_0's binary_logloss: 0.122443	valid_1's auc: 0.839214	valid_1's binary_logloss: 0.13927
[35]	valid_0's auc: 0.881022	valid_0's binary_logloss: 0.122197	valid_1's auc: 0.83935	valid_1's binary_logloss: 0.139237
[36]	valid_0's auc:

[34]	valid_0's auc: 0.879352	valid_0's binary_logloss: 0.122713	valid_1's auc: 0.837536	valid_1's binary_logloss: 0.139547
[35]	valid_0's auc: 0.879931	valid_0's binary_logloss: 0.122474	valid_1's auc: 0.837501	valid_1's binary_logloss: 0.139486
[36]	valid_0's auc: 0.881044	valid_0's binary_logloss: 0.122159	valid_1's auc: 0.837153	valid_1's binary_logloss: 0.139544
[37]	valid_0's auc: 0.881614	valid_0's binary_logloss: 0.121894	valid_1's auc: 0.83707	valid_1's binary_logloss: 0.139504
[38]	valid_0's auc: 0.882292	valid_0's binary_logloss: 0.121624	valid_1's auc: 0.837255	valid_1's binary_logloss: 0.139499
[39]	valid_0's auc: 0.882963	valid_0's binary_logloss: 0.121357	valid_1's auc: 0.837082	valid_1's binary_logloss: 0.139541
[40]	valid_0's auc: 0.883721	valid_0's binary_logloss: 0.121131	valid_1's auc: 0.8369	valid_1's binary_logloss: 0.139573
[41]	valid_0's auc: 0.884173	valid_0's binary_logloss: 0.120925	valid_1's auc: 0.837027	valid_1's binary_logloss: 0.139549
[42]	valid_0's auc:

[8]	valid_0's auc: 0.8514	valid_0's binary_logloss: 0.136369	valid_1's auc: 0.836779	valid_1's binary_logloss: 0.14653
[9]	valid_0's auc: 0.852973	valid_0's binary_logloss: 0.135111	valid_1's auc: 0.837232	valid_1's binary_logloss: 0.145427
[10]	valid_0's auc: 0.85477	valid_0's binary_logloss: 0.134	valid_1's auc: 0.839172	valid_1's binary_logloss: 0.144443
[11]	valid_0's auc: 0.857145	valid_0's binary_logloss: 0.132969	valid_1's auc: 0.839311	valid_1's binary_logloss: 0.143664
[12]	valid_0's auc: 0.858175	valid_0's binary_logloss: 0.132131	valid_1's auc: 0.838397	valid_1's binary_logloss: 0.143085
[13]	valid_0's auc: 0.859344	valid_0's binary_logloss: 0.131353	valid_1's auc: 0.838476	valid_1's binary_logloss: 0.142495
[14]	valid_0's auc: 0.860202	valid_0's binary_logloss: 0.130653	valid_1's auc: 0.838499	valid_1's binary_logloss: 0.142042
[15]	valid_0's auc: 0.861006	valid_0's binary_logloss: 0.13004	valid_1's auc: 0.838745	valid_1's binary_logloss: 0.141622
[16]	valid_0's auc: 0.8624

[15]	valid_0's auc: 0.860812	valid_0's binary_logloss: 0.130022	valid_1's auc: 0.837828	valid_1's binary_logloss: 0.141851
[16]	valid_0's auc: 0.861891	valid_0's binary_logloss: 0.129381	valid_1's auc: 0.838697	valid_1's binary_logloss: 0.141541
[17]	valid_0's auc: 0.863184	valid_0's binary_logloss: 0.128808	valid_1's auc: 0.83816	valid_1's binary_logloss: 0.141321
[18]	valid_0's auc: 0.864453	valid_0's binary_logloss: 0.128288	valid_1's auc: 0.837608	valid_1's binary_logloss: 0.1411
[19]	valid_0's auc: 0.86626	valid_0's binary_logloss: 0.127755	valid_1's auc: 0.838314	valid_1's binary_logloss: 0.140832
[20]	valid_0's auc: 0.867479	valid_0's binary_logloss: 0.127279	valid_1's auc: 0.838833	valid_1's binary_logloss: 0.140618
[21]	valid_0's auc: 0.868384	valid_0's binary_logloss: 0.126849	valid_1's auc: 0.838394	valid_1's binary_logloss: 0.140478
[22]	valid_0's auc: 0.869278	valid_0's binary_logloss: 0.126462	valid_1's auc: 0.838494	valid_1's binary_logloss: 0.140297
[23]	valid_0's auc: 

...

[20]	valid_0's auc: 0.865002	valid_0's binary_logloss: 0.128019	valid_1's auc: 0.837515	valid_1's binary_logloss: 0.140668
[21]	valid_0's auc: 0.866034	valid_0's binary_logloss: 0.127621	valid_1's auc: 0.837274	valid_1's binary_logloss: 0.140546
[22]	valid_0's auc: 0.866748	valid_0's binary_logloss: 0.127244	valid_1's auc: 0.837263	valid_1's binary_logloss: 0.140431
[23]	valid_0's auc: 0.867777	valid_0's binary_logloss: 0.126855	valid_1's auc: 0.837988	valid_1's binary_logloss: 0.140229
[24]	valid_0's auc: 0.86852	valid_0's binary_logloss: 0.126499	valid_1's auc: 0.838181	valid_1's binary_logloss: 0.140073
[25]	valid_0's auc: 0.869365	valid_0's binary_logloss: 0.126157	valid_1's auc: 0.837913	valid_1's binary_logloss: 0.140015
[26]	valid_0's auc: 0.870109	valid_0's binary_logloss: 0.125856	valid_1's auc: 0.837986	valid_1's binary_logloss: 0.139907
[27]	valid_0's auc: 0.871199	valid_0's binary_logloss: 0.125492	valid_1's auc: 0.838367	valid_1's binary_logloss: 0.139777
[28]	valid_0's au

[31]	valid_0's auc: 0.891103	valid_0's binary_logloss: 0.118763	valid_1's auc: 0.82532	valid_1's binary_logloss: 0.14198
[32]	valid_0's auc: 0.891922	valid_0's binary_logloss: 0.118414	valid_1's auc: 0.824702	valid_1's binary_logloss: 0.142096
[33]	valid_0's auc: 0.892665	valid_0's binary_logloss: 0.118057	valid_1's auc: 0.824663	valid_1's binary_logloss: 0.142137
[34]	valid_0's auc: 0.89333	valid_0's binary_logloss: 0.117732	valid_1's auc: 0.825178	valid_1's binary_logloss: 0.14201
[35]	valid_0's auc: 0.894088	valid_0's binary_logloss: 0.117417	valid_1's auc: 0.825489	valid_1's binary_logloss: 0.141931
[36]	valid_0's auc: 0.895211	valid_0's binary_logloss: 0.117066	valid_1's auc: 0.825124	valid_1's binary_logloss: 0.142049
Early stopping, best iteration is:
[6]	valid_0's auc: 0.855592	valid_0's binary_logloss: 0.138092	valid_1's auc: 0.834706	valid_1's binary_logloss: 0.14935
[1]	valid_0's auc: 0.828121	valid_0's binary_logloss: 0.155715	valid_1's auc: 0.818951	valid_1's binary_loglos

[18]	valid_0's auc: 0.877865	valid_0's binary_logloss: 0.124858	valid_1's auc: 0.834418	valid_1's binary_logloss: 0.141328
[19]	valid_0's auc: 0.87963	valid_0's binary_logloss: 0.124177	valid_1's auc: 0.834106	valid_1's binary_logloss: 0.141178
[20]	valid_0's auc: 0.880854	valid_0's binary_logloss: 0.123534	valid_1's auc: 0.834312	valid_1's binary_logloss: 0.140998
[21]	valid_0's auc: 0.882289	valid_0's binary_logloss: 0.122893	valid_1's auc: 0.834337	valid_1's binary_logloss: 0.140827
[22]	valid_0's auc: 0.883914	valid_0's binary_logloss: 0.122312	valid_1's auc: 0.83432	valid_1's binary_logloss: 0.140733
[23]	valid_0's auc: 0.88517	valid_0's binary_logloss: 0.121763	valid_1's auc: 0.834068	valid_1's binary_logloss: 0.140666
[24]	valid_0's auc: 0.886084	valid_0's binary_logloss: 0.12124	valid_1's auc: 0.833719	valid_1's binary_logloss: 0.140662
[25]	valid_0's auc: 0.887612	valid_0's binary_logloss: 0.120697	valid_1's auc: 0.832804	valid_1's binary_logloss: 0.140735
[26]	valid_0's auc: 

[1]	valid_0's auc: 0.828121	valid_0's binary_logloss: 0.155715	valid_1's auc: 0.818951	valid_1's binary_logloss: 0.165027
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.838452	valid_0's binary_logloss: 0.15013	valid_1's auc: 0.826575	valid_1's binary_logloss: 0.159601
[3]	valid_0's auc: 0.847999	valid_0's binary_logloss: 0.146048	valid_1's auc: 0.832463	valid_1's binary_logloss: 0.155895
[4]	valid_0's auc: 0.85237	valid_0's binary_logloss: 0.142812	valid_1's auc: 0.834669	valid_1's binary_logloss: 0.152926
[5]	valid_0's auc: 0.855337	valid_0's binary_logloss: 0.140257	valid_1's auc: 0.835732	valid_1's binary_logloss: 0.150601
[6]	valid_0's auc: 0.858629	valid_0's binary_logloss: 0.138031	valid_1's auc: 0.836283	valid_1's binary_logloss: 0.148801
[7]	valid_0's auc: 0.86144	valid_0's binary_logloss: 0.136075	valid_1's auc: 0.836287	valid_1's binary_logloss: 0.147288
[8]	valid_0's auc: 0.863399	valid_0's binary_logloss: 0.134391	valid_1's auc: 0.836116	v

KeyboardInterrupt: 

이것도 넘 오래 걸리니까 최적의 파라미터를 찾았다 가정하고! 그 파라미터로 실행했다 가정! (너무 오래 걸려 ㅜㅜ) 